In [10]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display
import json

In [32]:
class rbc_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                             param_dict['category'],
                             param_dict['dateFrom'],
                             param_dict['dateTo'],
                             param_dict['offset'],
                             param_dict['limit'],
                             param_dict['query'],
                             param_dict['material'])

        return url


    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()

        return search_table.sort_values('publish_date_t', ignore_index=True)


    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None

        return overview, text

    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        print("inited")
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0
        print("parse rdy")
        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table{}.xlsx".format(save_counter))
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
            with open("rbc_{}_{}.json".format(param_dict['dateFrom'],param_dict['dateTo']), 'w', encoding='utf-8') as file:
                out.to_json(file, orient='records', lines=True, force_ascii=False)

        print('Finish')

        return out
print(1)

1


In [33]:
# Задаем параметры запросы и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2022-09-01'
dateTo = "2022-10-07"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query,
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.09.2022', 'dateTo': '07.10.2022', 'offset': '0', 'limit': '100', 'material': ''}


In [34]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               includeText = True) # Парсить текст статей
print(len(tbl))
tbl.head()
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict=param_dict,
                            time_step = 1, # Шаг - 7 дней, можно больше,
                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                            save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                            save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Checkpoint saved!
Parsing articles from 01.10.2022 to 02.10.2022


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_10068\2846183737.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 03.10.2022 to 04.10.2022


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_10068\2846183737.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 05.10.2022 to 06.10.2022


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_10068\2846183737.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 07.10.2022 to 07.10.2022
Finish
190


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_10068\2846183737.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons,overview,text
0,634139269a79479a6431ead3,https://www.rbc.ru/business/08/10/2022/634139269a79479a6431ead3,1665219754,"Sat, 08 Oct 2022 12:02:34 +0300",Туроператоры оценили возможное количество туристов в Крыму,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/5/18/756652196781185.jpg'},None,Бизнес,None,None,"... в ближайшие дни должно заехать», — сказал он (цитата по ТАСС). [ <b>РБК</b> ] Взрыв и обрушение пролетов на Крымском мосту. Что известно к ... отдыха», сообщила в своем телеграмм-канале гла...","В общей сложности в Крыму сейчас находится до 50 тыс. туристов, считают в ассоциации туроператоров. Власти региона призвали гостиницы продлить им время проживания, а в Ростуризме посоветовали пере...",В Крыму сейчас может находиться порядка 50 тыс. туристов. Об этом говорится в сообщении Ассоциации туроператоров России. «Туроператоры и органы власти ведут их подсчет. Более точные данные о колич...
1,634135339a794798e76ecd97,https://www.rbc.ru/economics/08/10/2022/634135339a794798e76ecd97,1665220263,"Sat, 08 Oct 2022 12:11:03 +0300",Севастополь вслед за Крымом ввел ограничение на продажу бакалеи,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/9/48/756652181483489.jpg'},None,Экономика,None,[Полина Мартынова],"... , подсолнечного масла — на 46, а мясных консервов — на 106 дней. [ <b>РБК</b> ] Взрыв и обрушение пролетов на Крымском мосту. Что известно к ...","В Севастополе после взрыва на Крымском мосту и остановки сообщения ограничили отпуск бакалейных товаров до 3 кг или пачек в одни руки, чтобы избежать ажиотажа. Власти заверили, что в регионе пробл...","В Севастополе ограничили отпуск бакалейной продукции до 3 кг или пачек в одни руки, чтобы избежать ажиотажа на фоне проблем с сообщением с материковой частью России в связи с повреждением Крымског..."
2,63413f8e9a79479c4c7a4180,https://ufa.rbc.ru/ufa/08/10/2022/63413f8e9a79479c4c7a4180,1665220592,"Sat, 08 Oct 2022 12:16:32 +0300",«Уфаводоканал» проиграл еще одно дело против пивоваренной компании «Эфес»,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/9/03/756652205679039.jpg'},None,None,None,[Наиль Байназаров],"... и он встал на сторону «АБ Инбев Эфес». Как сообщал <b>РБК</b> Уфа, «Уфаводоканал» пытается взыскать с пивоваренного завода «Эфес» в Уфе ... сверх нормативов. Следите за деловыми новостями р...",Это дело имеет принципиальное значение для исхода других споров между ресурсоснабжающей организацией и пивоваренным заводом в Уфе,"Арбитражный суд Уральского округа оставил в силе решение 18-го арбитражного апелляционного суда, который отказал «Уфаводоканалу» в удовлетворении иска к АО «АБ Инбев Эфес». Ресурсоснабжающая компа..."
3,634140be9a79479c4c7a4188,https://www.rbc.ru/economics/08/10/2022/634140be9a79479c4c7a4188,1665220947,"Sat, 08 Oct 2022 12:22:27 +0300","Министр заявил, что запустить паромы в Крым в субботу не позволит погода",{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/9/81/756652212312819.jpg'},None,Экономика,None,[Полина Мартынова],"... , чтобы организовать регулярное паромное сообщение сегодня», — сказал и.о. министра. [ <b>РБК</b> ] Страховщики оценили стоимость ущерба из-за подрыва на Крымском мосту ...","Однако позднее глава крымского минтранса Николай Лукашенко заявил, что это будет возможно, суда уже тестируют","Запустить паромы в Крым в субботу не получится из-за погодных условий. Об этом заявил и.о. министра транспорта региона Николай Лукашенко. «Погода сегодня не позволяет вообще ничего», — заявил он. ..."
4,63413dd09a79479c1e5e3e6f,https://www.rbc.ru/politics/08/10/2022/63413dd09a79479c1e5e3e6f,1665220952,"Sat, 08 Oct 2022 12:22:32 +0300",Губернатор сообщил о втором за день обстреле села в Белгородской области,{'url': 'https://s0.rbk.ru/v6_top_pics/resized/250xH/media/img/4/01/756652205617014.jpg'},None,Политика,None,[Полина Мартынова],... селу Бирюч Валуйского городского округа. Там пострадала 16